In [1]:
# Choose a theme to your convenience
PLOTLY_THEME = "plotly_dark"
#PLOTLY_THEME = "plotly_white"


In [2]:
from dataclasses import dataclass, asdict

@dataclass
class SimulationParameters:
    re: float
    N: int
    steps: int
    sampling: int
    K: float
    dt: float
    scheme: str

In [3]:
import configparser

params = SimulationParameters(3200.0, 257, 50000, 5, 1.0, 1e-3, "IMEX")
# params = SimulationParameters(400.0, 129, 50000, 5, 1.0, 1e-3, "IMEX")

config = configparser.ConfigParser()
config["Simulation"] = asdict(params)
with open('simulation.ini', 'w') as configfile:
  config.write(configfile)

In [4]:
import subprocess,  os

def launch_c():
    subprocess.run(["cmake", "--build", "build"])
    
    with open(os.devnull, "w") as devnull:
        proc = subprocess.Popen(
            ["./build/cldc"],
            stdout=devnull,
            stderr=devnull
        )

In [5]:
import mmap
import struct
import time

from tqdm.notebook import tqdm

import numpy as np
import plot_run
import posix_ipc


FLAT_ARRAY_SIZE = params.N * params.N * 2
record_size = params.steps // params.sampling
output_array = np.zeros((record_size, params.N, params.N, 2))

shm = posix_ipc.SharedMemory("/sim_shm", posix_ipc.O_CREAT, size=8 + 8 * FLAT_ARRAY_SIZE)
mm = mmap.mmap(shm.fd, shm.size)

shm.close_fd()

mm.write(struct.pack("Q", 0))

write_idx = np.frombuffer(mm, dtype=np.uint64, count=1, offset=0)
data = np.frombuffer(mm, dtype=np.float64, count=FLAT_ARRAY_SIZE, offset=8)

plotter = plot_run.RunPlot(params.re, params.K, params.N, PLOTLY_THEME)
plotter.prepare()
last = 0
try:
    pbar = tqdm(total=params.steps)
    pbar.set_description("Waiting for C simulation")
    launch_c()
    while True:
        w = int(write_idx[0])
        index = w // params.sampling
        if w > last:
            pbar.set_description("")
            pbar.update(params.sampling)
            last = w
            frame = data.copy().reshape(params.N, params.N, 2)

            output_array[index] = frame
            plotter.update(frame, index)
        if index >= record_size - 1:
            break
        time.sleep(0.001)
except KeyboardInterrupt:
    pass
print("Saving run file")
np.save("runs/out", output_array)


FigureWidget({
    'data': [{'line': {'width': 2},
              'mode': 'lines',
              'name': 'min',
              'type': 'scatter',
              'uid': '54cc52ac-699e-4d44-9c86-7adad5e2f738',
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'width': 2},
              'mode': 'lines',
              'name': 'mean',
              'type': 'scatter',
              'uid': 'f69a35e4-acb3-4394-8bbd-51f8a37053d7',
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'width': 2},
              'mode': 'lines',
              'name': 'max',
              'type': 'scatter',
              'uid': '54c38782-7ee1-4b48-9e59-fed8c119f915',
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'width': 2},
              'mode': 'lines',
              'name': 'profile',
              'type': 'scatter',
              'uid': '31583e48-dc3e-4c2e-

  0%|          | 0/50000 [00:00<?, ?it/s]

[ 50%] Building C object CMakeFiles/cldc.dir/main.c.o


/home/kz/vault/all-cfd/CFD-Solver/C/main.c:589:9: missed: statement clobbers memory: _1 = strcmp (section_15(D), "Simulation");
/home/kz/vault/all-cfd/CFD-Solver/C/main.c:589:9: missed: statement clobbers memory: _2 = strcmp (name_16(D), "re");
/home/kz/vault/all-cfd/CFD-Solver/C/main.c:591:16: missed: statement clobbers memory: _3 = strcmp (name_16(D), "n");
/home/kz/vault/all-cfd/CFD-Solver/C/main.c:593:16: missed: statement clobbers memory: _4 = strcmp (name_16(D), "steps");
/home/kz/vault/all-cfd/CFD-Solver/C/main.c:595:16: missed: statement clobbers memory: _5 = strcmp (name_16(D), "sampling");
/home/kz/vault/all-cfd/CFD-Solver/C/main.c:597:16: missed: statement clobbers memory: _6 = strcmp (name_16(D), "k");
/home/kz/vault/all-cfd/CFD-Solver/C/main.c:599:16: missed: statement clobbers memory: _7 = strcmp (name_16(D), "dt");
/home/kz/vault/all-cfd/CFD-Solver/C/main.c:601:16: missed: statement clobbers memory: _8 = strcmp (name_16(D), "scheme");
/home/kz/vault/all-cfd/CFD-Solver/C/

[100%] Linking C executable cldc
[100%] Built target cldc
Saving run file
